In [ ]:
%matplotlib widget

In [ ]:
from glob import glob
import numpy as np
import pandas as pd
import flammkuchen as fl
from split_dataset import SplitDataset
from bouter import Experiment
from fimpy.pipeline.general import calc_f0, dff
from motions.utilities import stim_vel_dir_dataframe, quantize_directions
from scipy.interpolate import interp1d 
from scipy.signal import convolve2d
import colorspacious
import napari
import matplotlib.pyplot as plt
import json

from fimpylab.core.twop_experiment import TwoPExperiment
from fimpylab.core.lightsheet_experiment import LightsheetExperiment

from pathlib import Path

In [ ]:
# make sensory regressors. requires old bouter stimulus_param_log.
def make_sensory_regressors(exp, n_dirs=8, upsampling=5, sampling=1/3):
    stim = stim_vel_dir_dataframe(exp)
    bin_centres, dir_bins = quantize_directions(stim.theta)
    ind_regs = np.zeros((n_dirs, len(stim)))
    for i_dir in range(n_dirs):
        ind_regs[i_dir, :] = (np.abs(dir_bins - i_dir) < 0.1) & (stim.vel > 0.1)  

    dt_upsampled = sampling / upsampling
    t_imaging_up = np.arange(0, stim.t.values[-1], dt_upsampled)
    reg_up = interp1d(stim.t.values, ind_regs, axis=1, fill_value="extrapolate")(
        t_imaging_up
    )
    
    # 6s kernel
    u_steps = t_imaging_up.shape[0]
    u_time = np.arange(u_steps) * dt_upsampled
    decay = np.exp(-u_time / (1.5 / np.log(2)))
    kernel = decay / np.sum(decay)
    
    convolved = convolve2d(reg_up, kernel[None, :])[:, 0:u_steps]
    reg_sensory = convolved[:, ::upsampling]
    
    reg_up = reg_up[:, ::upsampling]

    return pd.DataFrame(reg_sensory.T, columns=[f"motion_{i}" for i in range(n_dirs)]), reg_up

In [ ]:
# find the frames to calculate the baseline.
def no_regressor_frames(regressors, threshold=0.01):
    return np.where(np.all(regressors.values < threshold, axis=1))[0]

# calculate the baseline, plane-wise
def calc_f0(stack, frames):
    fr_mean = None
    for i_frame in frames:
        sf = stack[int(i_frame), :, :]
        if fr_mean is None:
            fr_mean = sf
        else:
            fr_mean += sf
    return fr_mean / len(frames)

In [ ]:
master =  Path(r"Z:\Hagar\E0040\v31\post ablation")
fish_list = list(master.glob("*_f*"))

#fish_list = [Path(r"Z:\Hagar\crazy fish\230302_f0\e0040_v10"), Path(r"Z:\Hagar\crazy fish\230302_f2\e0040_v10"), Path(r"Z:\Hagar\crazy fish\230303_f1\e0040_v10")]

n_dirs = 8

In [ ]:
for fish in fish_list:
    print(fish)
    try:
        #if not (fish / "sensory_regressors.h5").exists():
        exp = glob(str(fish / "*behavior*"))[0]

        #traces = fl.load(fish / "filtered_traces.h5", "/detr")
        stack_file = str(fish / 'original' / "stack_metadata.json")
        with open(stack_file) as f:
            stack_metadata = json.load(f)

        len_rec = stack_metadata["shape_full"][0]

        exp_ls = LightsheetExperiment(fish)
        fs = int(exp_ls.fn)
        sampling = 1/fs
        time = np.linspace(0, len_rec*sampling, len_rec)

        stim = stim_vel_dir_dataframe(exp_ls)

        theta = np.asarray(stim.theta)

        bin_centres, dir_bins = quantize_directions(stim.theta)
        ind_regs = np.zeros((n_dirs, len(stim)))
        for i_dir in range(n_dirs):
            ind_regs[i_dir, :] = (np.abs(dir_bins - i_dir) < 0.1) & (stim.vel > 0.1) 

        #len_rec, num_cells = np.shape(traces)
        # make a list of sensory regressors 
        reg, reg_interp = make_sensory_regressors(Experiment(fish), sampling=sampling)
        reg_list = [reg]
        print(np.shape(reg))
        print(len_rec)

        d = {
            'regressors': reg,
            'theta': theta,
            'individual_theta': ind_regs,
            'individual_theta_interp': reg_interp,
        }
        fl.save(fish / 'sensory_regressors.h5', d)
    except:
        print("Error")